In [2]:
import pandas as pd
import numpy as ny
import findspark
import json
import nltk
import re
findspark.init()
from pyspark.sql import SparkSession

In [20]:
spark = SparkSession.builder.appName('metadata').getOrCreate()
df = spark.read.csv("metadata.csv")#, low_memory = False)
#reading in dataset in pyspark to see if it better or worse than pandas


In [21]:
df.columns #to make sure all data column names od not have any hidden spaces and for future reference

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14',
 '_c15',
 '_c16',
 '_c17',
 '_c18']

In [16]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)



In [22]:
pd.DataFrame(df.take(5), columns=df.columns)

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,_c15,_c16,_c17,_c18
0,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
1,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,None,None,None,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,None
2,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,None,None,None,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,None
3,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,None,None,None,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,None
4,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,None,None,None,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,None


In [27]:
pd.DataFrame(df.take(1))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id


In [28]:
df = df.rename(columns={'_c0':'cord_uid'})#, '_c1','_c2','_c3','_c4','_c5','_c6','_c7','_c8','_c9','_c10','_c11','_c12','_c13','_c14','_c15','_c16','_c17','_c18'})

AttributeError: 'DataFrame' object has no attribute 'rename'

In [30]:
dataF = pd.read_csv("metadata.csv", low_memory = False)

In [31]:
dataF.describe()

,pubmed_id,mag_id,s2_id
count,1.064600e+05,0.0,1.335230e+05
mean,2.563215e+07,NaN,1.301213e+08
std,7.335715e+06,NaN,9.572780e+07
min,2.142000e+03,NaN,9.600000e+01
25%,2.164183e+07,NaN,2.394433e+07
50%,2.742961e+07,NaN,2.080185e+08
75%,3.226864e+07,NaN,2.185324e+08
max,3.256221e+07,NaN,2.198834e+08


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158942,mixxjvte,2166cf20184f1b770a1a6f75682052fcfa42b396,Medline; PMC,Setting out on a fantastic voyage to advance n...,10.1038/s42003-020-0943-z,PMC7192934,32355225.0,cc-by,"We are inviting submissions of articles, persp...",2020-04-30,NaN,Commun Biol,NaN,NaN,NaN,document_parses/pdf_json/2166cf20184f1b770a1a6...,document_parses/pmc_json/PMC7192934.xml.json,https://doi.org/10.1038/s42003-020-0943-z; htt...,216650000.0
158943,t9bt70f2,d2a557f18ca3ad25962a7fce457eff2802f2adaa; a051...,BioRxiv; MedRxiv,Host and infectivity prediction of Wuhan 2019 ...,10.1101/2020.01.21.914044,NaN,NaN,medrxiv,"The recent outbreak of pneumonia in Wuhan, Chi...",2020-01-24,"Guo, Qian; Li, Mo; Wang, Chunhui; Wang, Peihon...",bioRxiv,NaN,NaN,NaN,document_parses/pdf_json/d2a557f18ca3ad25962a7...,NaN,https://doi.org/10.1101/2020.01.21.914044,213133457.0
158944,nhpjvgt4,da516a5f665fcb8a6f3a1fd350bcfa34820d951a,Medline; PMC,Considerations for Safety in the Use of System...,10.1111/dth.13687,PMC7283778,32458536.0,no-cc,"Coronavirus disease 2019 (COVID‐19), is respon...",2020-05-27,"Ricardo, Jose W.; Lipner, Shari R.",Dermatol Ther,NaN,NaN,NaN,document_parses/pdf_json/da516a5f665fcb8a6f3a1...,NaN,https://doi.org/10.1111/dth.13687; https://www...,218911875.0
158945,8gjng3o9,926208e67493a6e5ae8d770cae1fe98f3116f678,Elsevier; Medline; PMC,Wearing masks and the fight against the novel ...,10.1016/j.pulmoe.2020.05.011,PMC7256497,32507498.0,no-cc,NaN,2020-05-29,"Alizargar, Javad",Pulmonology,NaN,NaN,NaN,document_parses/pdf_json/926208e67493a6e5ae8d7...,document_parses/pmc_json/PMC7256497.xml.json,https://www.sciencedirect.com/science/article/...,218973065.0
